In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
from os.path import join as pjoin
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append("../..")
import circletrack_behavior as ctb
import plotting_functions as pf

In [2]:
## Set parameters
behavior_path = '../../../MultiCon_AfterHours/'
num_experiments = 2 + 1 ## change first value since np.arange is not inclusive
experiment_list = ['MultiCon_EEG{}'.format(x) for x in np.arange(1, num_experiments)]
session_dict = {'mc_EEG1_01': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5'],
                'mc_EEG1_02': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'A2_1', 'A2_2', 'A2_3', 'A2_4', 'A2_5'],
                'mc_EEG2_02': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5',
                               'C1', 'C2', 'C3', 'C4', 'C5', 'A2_1', 'A2_2', 'A2_3', 'A2_4', 'A2_5']}

In [3]:
## Calculate lick accuracy, dprime metrics
experiment_pc = {}
experiment_pc_trial = {}
experiment_sdt = {}
for experiment in experiment_list:
    bpath = pjoin(behavior_path, experiment)
    bpath = pjoin(bpath, 'output/behav')
    mouse_pc = {}
    mouse_pc_trial = {}
    mouse_sdt = {}
    for mouse in os.listdir(bpath):
        path = pjoin(bpath, mouse)
        percent_correct = {}
        percent_correct_trials = {}
        sdt_metrics = {}
        for session in np.arange(0, len(session_dict[mouse])):
            behav = pd.read_feather(pjoin(path, '{}_{}.feat'.format(mouse, session_dict[mouse][session])))
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            forward_trials, reverse_trials = ctb.get_forward_reverse_trials(behav)
            forward_df = behav.loc[(behav['trials'].isin(forward_trials))]
            ## Percent correct licking
            pc = ctb.lick_accuracy(forward_df, port_one=reward_one, port_two=reward_two, by_trials=False)
            pc_trials = ctb.lick_accuracy(forward_df, port_one=reward_one, port_two=reward_two, by_trials=True)
            percent_correct[session] = pc
            percent_correct_trials[session] = pc_trials
            ## dprime, correct rejection, hit rate
            metrics = ctb.dprime_metrics(forward_df, reward_one, reward_two, reward_index='one', forward_reverse='forward')
            sdt_metrics[session] = metrics
        mouse_pc[mouse] = percent_correct
        mouse_pc_trial[mouse] = percent_correct_trials
        mouse_sdt[mouse] = sdt_metrics
    experiment_pc[experiment] = mouse_pc
    experiment_pc_trial[experiment] = mouse_pc_trial
    experiment_sdt[experiment] = mouse_sdt

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:989: RuntimeWarning: invalid value encountered in long_scalars
  percent_correct.append(((count_licks['first_licks'][(count_licks['lick_port'] == port_one) | (count_licks['lick_port'] == port_two)].dropna().sum()) /
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:989: RuntimeWarning: invalid value encountered in long_scalars
  percent_correct.append(((count_licks['first_licks'][(count_licks['lick_port'] == port_one) | (count_licks['lick_port'] == port_two)].dropna().sum()) /
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_AfterHours/analysis/../../circletrack_behavior.py:989: RuntimeWarning: invalid value encountered in long_scalars
  percent_correct.append(((count_licks['first_licks'][(count_licks['lick_port'] == port_one) | (count_licks['lick_port'] == por

In [26]:
## Percent correct data frame for graphing
accuracy = pd.DataFrame()
for experiment in experiment_pc:
    for mouse in experiment_pc[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_pc[experiment][mouse], index=[0]).T.rename(columns={0: 'percent_correct'})
            df.insert(0, 'mouse', mouse)
            df.insert(1, 'day', [x for x in np.arange(1, 21)])
            accuracy = pd.concat([accuracy, df], axis=0, ignore_index=True)

fig = pf.plot_behavior_across_days(accuracy, x_var='day', y_var='percent_correct', groupby_var=['day'], title='Lick Accuracy',
                                   x_title='Day', y_title='Percent Correct (%)', template='simple_white')
fig.show()

In [25]:
## d' across days
dprime_df = pd.DataFrame()
for experiment in experiment_pc:
    for mouse in experiment_pc[experiment]:
        if mouse == 'mc_EEG2_01':
            next
        else:
            df = pd.DataFrame(experiment_sdt[experiment][mouse]).T
            d = []
            for idx in np.arange(0, len(df['dprime'])):
                d.append(np.nanmean(df['dprime'][idx]))
            dprime = pd.DataFrame(d).rename(columns={0: 'dprime'})
            dprime.insert(0, 'mouse', mouse)
            dprime.insert(1, 'day', [x for x in np.arange(1, 21)])
            dprime_df = pd.concat([dprime_df, dprime], axis=0, ignore_index=True)

fig = pf.plot_behavior_across_days(dprime_df, x_var='day', y_var='dprime', groupby_var=['day'], title='',
                                   x_title='Day', y_title="d'", template='simple_white')
fig.show()

/tmp/ipykernel_40835/4190553665.py:11: RuntimeWarning:

Mean of empty slice



In [27]:
fig = pf.plot_lick_raster(behav, title='Lick Raster', x_title='Linearized Position (cm)', y_title='Trial')
fig.show()

In [28]:
fig = pf.plot_circle_position(behav, title='', x_title='Position (x)', y_title='Position (y)')
fig.show()